In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

In [ ]:
# Download YOLOv7 repository and install requirements
#!git clone https://github.com/WongKinYiu/yolov7
#%cd yolov5
#!pip install -r requirements.txt

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 5.3 MB/s 
Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

     |████████████████████████████████| 42 kB 765 kB/s 
     |████████████████████████████████| 178 kB 9.4 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 145 kB 71.0 MB/s 
     |████████████████████████████████| 138 kB 71.3 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=roboflow-yolov5


In [ ]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="zkyt15pyUkXSH9tlso0C")
project = rf.workspace("noor-djz41").project("multiuav")
dataset = project.version(4).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to MultiUav-3 in yolov5pytorch:: 100%|██████████| 23478/23478 [00:04<00:00, 5189.21it/s] 


In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names:
- Fixed Wings
- Multi-rotor
- Single rotor
nc: 3
roboflow:
  license: CC BY 4.0
  project: multiuav
  url: https://universe.roboflow.com/noor-djz41/multiuav/dataset/3
  version: 3
  workspace: noor-djz41
test: ../test/images
train: MultiUav-3/train/images
val: MultiUav-3/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/hub/yolov5s-transformer.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3TR, [1024]],  # 9 <--- C3TR() Transformer module
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'ne

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/YOLOv5-CVIT-FPN.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32
# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, CrossConv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3x, [128]],
   [-1, 1, CrossConv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3x, [256]],
   [-1, 1, CrossConv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3x, [512]],
   [-1, 1, CrossConv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3TR, [1024]],  # 9 <--- C3TR() Transformer module
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 FPN head
head:
  [[-1, 3, C3x, [1024, False]],  # 10 (P5/32-large)

   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 1, CrossConv, [512, 1, 1]],
   [-1, 3, C3x, [512, False]],  # 14 (P4/16-medium)

   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 1, CrossConv, [256, 1, 1]],
   [-1, 3, C3x, [256, False]],  # 18 (P3/8-small)

   [[18, 14, 10], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# train yolov5s on custom data epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch-size -1 --optimizer AdamW --label-smoothing 0.01 --cos-lr --patience 50 --epochs 1000 --hyp data/hyps/hyp.scratch-low.yaml --data {dataset.location}/data.yaml --cfg ./models/YOLOv5-CVIT-FPN.yaml --project "/content/gdrive/MyDrive/Runs"  --weights '' --name YOLOv5-CVIT-FPN_results  --cache

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
!python train.py --img 416 --weights '' --project "/content/gdrive/MyDrive/Runs"  --name YOLOv5-CVIT-FPN_results  --cache  --resume "/content/gdrive/MyDrive/Runs/YOLOv5-CVIT-FPN_results/weights/last.pt"

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
#########Validate whole dataset
# Validate model and print evaluation metrics
!python val.py --weights runs/train/YOLOv5-CVIT-FPN_results/weights/best.pt --data {dataset.location}/data.yaml --img 416--half

In [ ]:
######## Test Valid image folder
%cd /content/yolov5/
!python detect.py --weights runs/train/YOLOv5-CVIT-FPN_results/weights/best.pt --img 416 --conf 0.25 --source /content/yolov5/MultiUav-3/valid/images

In [ ]:
!python val.py --task study --weights runs/train/YOLOv5-CVIT-FPN_results/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
!python val.py --task speed --weights runs/train/YOLOv5-CVIT-FPN_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
##I have created a zip file:
!zip -r /content/YOLOv5-CVIT-FPN.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/YOLOv5-CVIT-FPN.zip")